In [ ]:
%matmodlab

# Von Mises Material Optimization

Optimize the values of Young's modulus, yield strength, and linear hardening parameters for a linear hardening Von Mises material model using data from a standard uniaxial tension test. The data represents the response of Aluminum, alloy unkown.

The strategy is to read read data from an excel file and use the axial strain to drive the simulation. Optimization is performed by minimizing the area between the stress-strain curve calculated and the measured stress-strain curve.

In [ ]:
# get the experimental stress and strain
filename = os.path.join(_dh[0], 'optimize.xls')
assert os.path.isfile(filename)
data = loadfile(filename, sheetname='MML', disp=0, columns=['E.XX', 'S.XX'])

In [ ]:
def runjob(job, names, x):
    """Run a specific MaterialPointSimulator job for parameters in x"""
    xp = dict(zip(names, x))
    NU = 0.32  # poisson's ratio for aluminum
    parameters = {'K': xp['E']/3.0/(1.0-2.0*NU), 
                  'G': xp['E']/2.0/(1.0+NU),
                  'Y0': xp['Y0'], 
                  'H': xp['H'], 
                  'BETA': 0.0}
    mps = MaterialPointSimulator(job)
    mps.Material('vonmises', parameters)
    mps.DataSteps(filename, steps=30, sheetname='MML',
                  columns=('E.XX',), descriptors='ESS')
    return mps

In [ ]:
def func(x, names, evald, job, *args):
    """The objective function"""
    mps = runjob(job, names, x)
    if mps.failed:
        return 1.0e9
    e, s = mps.get('E.XX', 'S.XX')
    error = calculate_bounded_area(data[:,0], data[:,1], e, s)
    return error

# set the variables to be optimized
E = OptimizeVariable('E',  2.0e6, bounds=(1.0e5, 1.0e7))
Y0= OptimizeVariable('Y0', 0.3e5, bounds=(1.0e4, 1.0e6))
H = OptimizeVariable('H',  1.0e6, bounds=(1.0e4, 1.0e7))
xinit = [E, Y0, H]

# the optimzer object
optimizer = Optimizer('optimize', func, xinit, method=SIMPLEX,
                      maxiter=200, tolerance=1.e-3)

# run the job
optimizer.run()

## Check the Results

In [ ]:
mps = runjob('Optimized', optimizer.names, optimizer.xopt)
plot = create_figure(x_axis_label='E.XX', y_axis_label='S.XX')
plot.scatter(data[:,0], data[:,1], marker='x', color='red', legend='Experimental data')
mps.plot('E.XX', 'S.XX', legend='Simulation', color='black', plot=plot, line_width=3)
show(plot)